<a href="https://colab.research.google.com/github/Gauri-44/FDS-LAB-PRACTICALS-/blob/main/Gauri_V_Korgaonkar_FDS_07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import warnings as wr
wr.filterwarnings("ignore")

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (confusion_matrix, classification_report, roc_auc_score,precision_recall_curve, f1_score, auc)
from imblearn.over_sampling import SMOTE
from sklearn.pipeline import Pipeline
from sklearn.calibration import CalibratedClassifierCV

In [ ]:
import pandas as pd
# Download dataset
!kaggle datasets download -d mlg-ulb/creditcardfraud
# Unzip dataset
!unzip creditcardfraud.zip
# Load the file into pandas
data1 = pd. read_csv('creditcard.csv')

Dataset URL: https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud
License(s): DbCL-1.0
creditcardfraud.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  creditcardfraud.zip
replace creditcard.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: Yes
  inflating: creditcard.csv          


In [ ]:
data1.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [ ]:
X = data1.drop('Class', axis=1)
y = data1['Class']


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2, random_state=42)
# Apply SMOTE to balance the data
smote = SMOTE(random_state=42)
X_train_sm, y_train_sm = smote.fit_resample(X_train, y_train)

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
pipeline = Pipeline([
       ('scaler', StandardScaler()),
       ('classifier', RandomForestClassifier(n_estimators=50, random_state=42))
   ])

In [ ]:
pipeline.fit(X_train_sm, y_train_sm)
y_pred = pipeline.predict(X_test)

In [ ]:
print("Confusion Matrix:\n", confusion_matrix(y_test,y_test))
print("Classification Report:\n", classification_report(y_test, y_pred))
#roc_auc = roc_auc_score(y_test, pipeline.predict_proba(X_test)[:, 1])
#print("ROC-AUC Score:", roc_auc)
precision, recall, _ = precision_recall_curve(y_test, pipeline.predict_proba(X_test)[:, 1])
pr_auc = auc(recall, precision)
print("PR-AUC Score:", pr_auc)

Confusion Matrix:
 [[56864     0]
 [    0    98]]
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     56864
           1       0.86      0.85      0.86        98

    accuracy                           1.00     56962
   macro avg       0.93      0.92      0.93     56962
weighted avg       1.00      1.00      1.00     56962

PR-AUC Score: 0.8661655889247794


In [20]:
calibrated_clf = CalibratedClassifierCV(pipeline.named_steps['classifier'],method= 'isotonic', cv='prefit')
calibrated_clf.fit(X_train_sm, y_train_sm)
y_calibrated_pred = calibrated_clf.predict(X_test)

In [21]:

# Print calibrated predictions
print("Calibrated Predictions:", y_calibrated_pred)

# Optionally, compare with original predictions
print("Original Predictions:", y_pred)

# Print a classification report for calibrated predictions
print("Classification Report (Calibrated):\n", classification_report(y_test, y_calibrated_pred))

# Print the confusion matrix for calibrated predictions
print("Confusion Matrix (Calibrated):\n", confusion_matrix(y_test, y_calibrated_pred))

Calibrated Predictions: [0 0 0 ... 0 0 0]
Original Predictions: [1 0 0 ... 0 0 0]
Classification Report (Calibrated):
               precision    recall  f1-score   support

           0       1.00      0.92      0.96     56864
           1       0.01      0.53      0.02        98

    accuracy                           0.92     56962
   macro avg       0.51      0.73      0.49     56962
weighted avg       1.00      0.92      0.96     56962

Confusion Matrix (Calibrated):
 [[52327  4537]
 [   46    52]]
